# © Artur Czarnecki. All rights reserved.
# Integrax framework – proprietary and confidential.
# Use, modification, or distribution without written permission is prohibited.


In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..")))

# Dowload video and audio from YoutTube

In [ ]:
from intergrax.multimedia.video_loader import yt_download_video
from intergrax.multimedia.audio_loader import yt_download_audio

url = "https://www.youtube.com/watch?v=eur8dUO9mvE"
out_dir = "video/video02"

video_path = yt_download_video(url, out_dir)
audio_path = yt_download_audio(url, out_dir)
print("Video saved to: ", video_path)
print("Audio saved to: ", audio_path)

# Transcribe video to vtt format

In [ ]:
from intergrax.multimedia.video_loader import transcribe_to_vtt

vtt_path = transcribe_to_vtt(video_path, model_size="base", language=None)
print("VTT saved to: ", vtt_path)

# Extract frames and metadatas from video

In [ ]:
from intergrax.multimedia.video_loader import extract_frames_and_metadata
import os
from pathlib import Path

extracted_frames_path = os.path.join(out_dir, "extracted_frame")
metadatas_path = out_dir

Path(out_dir).mkdir(parents=True, exist_ok=True)
Path(extracted_frames_path).mkdir(parents=True, exist_ok=True)

metadatas = extract_frames_and_metadata(
    path_to_video=str(video_path),
    path_to_transcript=str(vtt_path),
    path_to_save_extracted_frames=extracted_frames_path,
    path_to_save_metadatas=metadatas_path)

print(metadatas)

# Translate audio

In [ ]:
from intergrax.multimedia.audio_loader import translate_audio

results = translate_audio(str(audio_path))
path_to_generated_trans = os.path.join(out_dir, "generated_translation.vtt")
with open(path_to_generated_trans, "w") as f:
    f.write(results['text'])

print(results['text'])

# Use ollama model to describe image

In [ ]:
from intergrax.multimedia.images_loader import transcribe_image
from IPython.display import Image, display

!ollama pull llava-llama3:latest

path_to_frame = os.path.join(extracted_frames_path, "frame_58.jpg")
display(Image(path_to_frame))
prompt = "Describe this image in details."

response = transcribe_image(prompt=prompt, image_path=path_to_frame)
print(response)



# Extract frame from video and transcribe images

In [ ]:
from intergrax.multimedia.video_loader import yt_download_video
from intergrax.multimedia.video_loader import extract_frames_from_video
import os
from pathlib import Path
from tqdm.auto import tqdm
from intergrax.multimedia.images_loader import transcribe_image
from IPython.display import Image, display

url = "https://www.youtube.com/watch?v=eur8dUO9mvE"
out_dir = "video/video02"

video_path = yt_download_video(url, out_dir)
print("Video saved to: ", video_path)

extracted_frames_path = os.path.join(out_dir, "extracted_frame")
metadatas_path = out_dir

Path(out_dir).mkdir(parents=True, exist_ok=True)
Path(extracted_frames_path).mkdir(parents=True, exist_ok=True)

paths = extract_frames_from_video(
    path_to_video=video_path,
    path_to_save_extracted_frames=extracted_frames_path,
    every_seconds=1.0,    
)
print(f"Saved {len(paths)} frames.")

prompt = "Describe this image in details."

for frame in tqdm(paths, desc="Translating frames", total=len(paths)):
    print(frame)
    display(Image(frame))
    print(transcribe_image(prompt=prompt, image_path=frame))